# Sales Report using SQL & Pandas

In [10]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd 
import plotly.express as px 
from fpdf import FPDF

## Paths and chart style

In [11]:
# "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "plotly_dark"

In [12]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

## Total Sales by Month

In [13]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [14]:
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)
print(df)

      sale_date  total_sales
0    2022-01-01          400
1    2022-01-03          945
2    2022-01-04          475
3    2022-01-05          175
4    2022-01-06         1050
..          ...          ...
340  2022-12-27         1525
341  2022-12-28          200
342  2022-12-29          180
343  2022-12-30         2075
344  2022-12-31         1800

[345 rows x 2 columns]


In [15]:
query = '''
    SELECT * FROM sales
    WHERE total_price > 100
'''

df_having = pd.read_sql_query(query, conn)
df_having.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_id      829 non-null    int64 
 1   sale_date    829 non-null    object
 2   customer_id  829 non-null    int64 
 3   product_id   829 non-null    int64 
 4   quantity     829 non-null    int64 
 5   unit_price   829 non-null    int64 
 6   total_price  829 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 45.5+ KB


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    345 non-null    object
 1   total_sales  345 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.5+ KB


In [17]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    345 non-null    datetime64[ns]
 1   total_sales  345 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.5 KB


In [18]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2022-01-01,400
2022-01-03,945
2022-01-04,475


In [19]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

,total_sales
sale_date,
2022-01-31,19285
2022-02-28,17180
2022-03-31,23135
2022-04-30,22670
2022-05-31,22935
2022-06-30,21910
2022-07-31,26265
2022-08-31,30140
2022-09-30,28260


In [20]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,19285,Jan
2022-02-28,17180,Feb
2022-03-31,23135,Mar
2022-04-30,22670,Apr
2022-05-31,22935,May
2022-06-30,21910,Jun
2022-07-31,26265,Jul
2022-08-31,30140,Aug
2022-09-30,28260,Sep


In [21]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
    x='month_name',
    y='total_sales',
    template=plotly_template,
    text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

In [22]:
# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png',
                width=1920,
                height=1080,
                scale=1)

## Total Sales by Product

In [23]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.name
'''
df_sales = pd.read_sql_query(query, conn)

In [24]:
df_sales

,name,total_sales
0,Product A,62600
1,Product B,28400
2,Product C,76125
3,Product D,42440
4,Product E,69960


In [25]:
# Create the Plotly figure with text parameter
fig = px.bar(df_sales,
    x='name',
    y='total_sales',
    template=plotly_template,
    text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Products',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png',
                width=1920,
                height=1080,
                scale=1)

## Top Customer by Sales

In [26]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.f_name || ' ' || c.l_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df_customer = pd.read_sql_query(query, conn)

In [27]:
df_customer

,customer_name,total_sales
0,Grace Lee,31645
1,David Brown,31475
2,Alice Jones,30845
3,Frank Wilson,30325
4,John Doe,27220
5,Bob Smith,27170
6,Henry Chen,26185
7,Isabel Garcia,25320
8,Emily Davis,24785
9,Jane Doe,24555


In [28]:
# Create the Plotly figure
fig = px.bar(df_customer,
    x='customer_name',
    y='total_sales',
    template=plotly_template,
    text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1920,
                height=1080,
                scale=1)

## Create PDF Report

In [35]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

''

## Bonus - SQL vs. Pandas

### Customer Segment using SQL

In [30]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.f_name || ' ' || customers.l_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales,customer_segment
0,8,Grace Lee,31645,High Value
1,5,David Brown,31475,High Value
2,4,Alice Jones,30845,High Value
3,7,Frank Wilson,30325,High Value
4,1,John Doe,27220,Medium Value
5,3,Bob Smith,27170,Medium Value
6,9,Henry Chen,26185,Medium Value
7,10,Isabel Garcia,25320,Low Value
8,6,Emily Davis,24785,Low Value
9,2,Jane Doe,24555,Low Value


### Customer Segment using SQL + Pandas

In [31]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.f_name || ' ' || customers.l_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales
0,1,John Doe,27220
1,2,Jane Doe,24555
2,3,Bob Smith,27170
3,4,Alice Jones,30845
4,5,David Brown,31475
5,6,Emily Davis,24785
6,7,Frank Wilson,30325
7,8,Grace Lee,31645
8,9,Henry Chen,26185
9,10,Isabel Garcia,25320


In [32]:
# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df

,customer_id,customer_name,total_sales,customer_segment
7,8,Grace Lee,31645,High Value
4,5,David Brown,31475,High Value
3,4,Alice Jones,30845,High Value
6,7,Frank Wilson,30325,High Value
0,1,John Doe,27220,Medium Value
2,3,Bob Smith,27170,Medium Value
8,9,Henry Chen,26185,Medium Value
9,10,Isabel Garcia,25320,Low Value
5,6,Emily Davis,24785,Low Value
1,2,Jane Doe,24555,Low Value


In [33]:
# Close the connection
conn.close()